# Section 0: Importing libraries

In [49]:
import pandas as pd

# Section 1: Data pre-processing
---
[Loading the data](#loading-the-data) \
[Dataset properties](#data-properties)

### Loading the data

In [50]:
# main data path and relative paths to the datasets
DATA_PATH = "./data/"
DATASET_PATH = {
    "movie_metadata": DATA_PATH + "movie.metadata.tsv",
    "character_metadata": DATA_PATH + "character.metadata.tsv",
    "plot_summaries": DATA_PATH + "plot_summaries.txt",
}

In [51]:
# columns used when loading a specific dataset (for the attribute "names" in pandas.read_csv)
DATASET_COLUMNS = {
    "movie_metadata": ["wikipedia_id", "freebase_movie_id", "name", "release_date", "revenue", "runtime", "languages", "countries", "genres"],
    "character_metadata": ["wikipedia_id", "freebase_movie_id", "release_date", "character_name", "birthday", "gender", "height", "ethnicity", "name", "age_at_release", "freebase_map", "freebase_char_id", "freebase_actor_id"]
}

In [52]:
# load the datasets as pandas dataframes
movie_metadata = pd.read_csv(DATASET_PATH["movie_metadata"], sep='\t', names=DATASET_COLUMNS["movie_metadata"], index_col=False, header=None)
character_metadata = pd.read_csv(DATASET_PATH["character_metadata"], sep='\t', names=DATASET_COLUMNS["character_metadata"], index_col=False, header=None)

### Dataset properties

In [53]:
print("Number of movie rows: ", movie_metadata.shape[0])
print("Number of features per movie: ", movie_metadata.shape[1])
# a random movie sample
movie_metadata.sample(1)

Number of movie rows:  81741
Number of features per movie:  9


,wikipedia_id,freebase_movie_id,name,release_date,revenue,runtime,languages,countries,genres
41788,3320322,/m/095mql,Sixth Happiness,1997,NaN,97.0,{},"{""/m/07ssc"": ""United Kingdom""}","{""/m/07s9rl0"": ""Drama"", ""/m/01f9r0"": ""Docudram..."


In [54]:
print("Number of character rows: ", character_metadata.shape[0])
print("Number of features per character: ", character_metadata.shape[1])
# a random character sample
character_metadata.sample(1)

Number of character rows:  450669
Number of features per character:  13


,wikipedia_id,freebase_movie_id,release_date,character_name,birthday,gender,height,ethnicity,name,age_at_release,freebase_map,freebase_char_id,freebase_actor_id
217138,35861350,/m/0jwschy,1927-10-21,NaN,1892-01-19,F,1.65,NaN,Mady Christians,NaN,/m/0mzndcj,NaN,/m/039m2r
